# PCam Small CNN

## 1) Optuna

In [1]:
import optuna
import torch
from torch import nn, optim
from src.datasets.dataloaders import get_pcam_dataloaders
from src.models.small_cnn_gpu import SmallCNN
from src.training.utils_training import evaluate_binary_classifier

# 1. Setup
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DATA_ROOT = "data/raw"

def objective(trial):
    # Hyperparameter-Vorschläge
    lr = trial.suggest_float("lr", 1e-4, 5e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [64, 128])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    
    # Dataloader für Tuning (begrenztes Set für Speed)
    # limit_per_split sorgt dafür, dass die Trials schnell gehen
    loaders = get_pcam_dataloaders(
        data_root=DATA_ROOT,
        batch_size=batch_size,
        num_workers=0, # Im Notebook auf Windows sicherheitshalber 0
        limit_per_split=20000 # ca. 8-10% des Datensatzes
    )
    
    model = SmallCNN(dropout_p=0.2).to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    criterion = nn.BCEWithLogitsLoss()
    
    # Kurzes Training für das Tuning (z.B. 5 Epochen)
    for epoch in range(5):
        model.train()
        for images, labels in loaders["train"]:
            images, labels = images.to(DEVICE), labels.float().to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(images), labels)
            loss.backward()
            optimizer.step()
            
    # Evaluation auf dem Validierungs-Set
    _, val_auroc, _ = evaluate_binary_classifier(model, loaders["val"], criterion, DEVICE)
    return val_auroc

# Studie starten
study_cnn = optuna.create_study(direction="maximize")
study_cnn.optimize(objective, n_trials=15)

print("✅ Tuning fertig!")
print(f"Beste AUROC: {study_cnn.best_value:.4f}")
print(f"Beste Parameter: {study_cnn.best_params}")

[I 2025-12-25 01:13:42,670] A new study created in memory with name: no-name-2179e543-ec6b-4ea9-8c2f-b4b9ea508b50
[I 2025-12-25 01:16:55,955] Trial 0 finished with value: 0.887096838709864 and parameters: {'lr': 0.0003843120994544614, 'batch_size': 64, 'weight_decay': 1.8765914810796102e-06}. Best is trial 0 with value: 0.887096838709864.
[I 2025-12-25 01:20:05,877] Trial 1 finished with value: 0.8986096719819521 and parameters: {'lr': 0.0011692741755619618, 'batch_size': 128, 'weight_decay': 3.481434751802591e-05}. Best is trial 1 with value: 0.8986096719819521.
[I 2025-12-25 01:23:17,125] Trial 2 finished with value: 0.8839527346234031 and parameters: {'lr': 0.00018885107643861452, 'batch_size': 128, 'weight_decay': 1.1462949543600632e-06}. Best is trial 1 with value: 0.8986096719819521.
[I 2025-12-25 01:26:25,966] Trial 3 finished with value: 0.8802783790045154 and parameters: {'lr': 0.0014126816776050264, 'batch_size': 128, 'weight_decay': 6.23321038916348e-05}. Best is trial 1 wit

✅ Tuning fertig!
Beste AUROC: 0.8986
Beste Parameter: {'lr': 0.0011692741755619618, 'batch_size': 128, 'weight_decay': 3.481434751802591e-05}


## 2) Training

In [2]:
from pathlib import Path

def train_small_cnn_final(best_params, run_name, num_epochs=20):
    print(f"\n🚀 Starte finales Training: {run_name}")
    
    # Full Dataloader (ohne Limit!)
    loaders = get_pcam_dataloaders(
        data_root=DATA_ROOT,
        batch_size=best_params["batch_size"],
        num_workers=0, 
        pin_memory=True
    )
    
    model = SmallCNN(dropout_p=0.2).to(DEVICE)
    optimizer = optim.Adam(
        model.parameters(), 
        lr=best_params["lr"], 
        weight_decay=best_params["weight_decay"]
    )
    criterion = nn.BCEWithLogitsLoss()
    
    best_val_auprc = 0.0
    save_path = Path("experiments/final_models")
    save_path.mkdir(parents=True, exist_ok=True)

    for epoch in range(1, num_epochs + 1):
        model.train()
        train_loss = 0.0
        for images, labels in loaders["train"]:
            images, labels = images.to(DEVICE), labels.float().to(DEVICE)
            optimizer.zero_grad()
            logits = model(images)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * images.size(0)

        # Evaluation
        avg_train_loss = train_loss / len(loaders["train"].dataset)
        val_loss, val_auroc, val_auprc = evaluate_binary_classifier(
            model, loaders["val"], criterion, DEVICE
        )

        print(f"[{epoch:02d}/{num_epochs}] Train Loss: {avg_train_loss:.4f} | Val AUROC: {val_auroc:.4f} | Val AUPRC: {val_auprc:.4f}")

        # Speichern des besten Modells (nach AUPRC, wie im Feedback empfohlen)
        if val_auprc > best_val_auprc:
            best_val_auprc = val_auprc
            torch.save(model.state_dict(), save_path / f"{run_name}_best.pth")
            print(f"⭐ Modell gespeichert!")

# Ausführen mit den Ergebnissen von oben
train_small_cnn_final(study_cnn.best_params, "small_cnn_final_baseline")


🚀 Starte finales Training: small_cnn_final_baseline
[01/20] Train Loss: 0.4201 | Val AUROC: 0.8956 | Val AUPRC: 0.8912
⭐ Modell gespeichert!
[02/20] Train Loss: 0.3647 | Val AUROC: 0.8314 | Val AUPRC: 0.8673
[03/20] Train Loss: 0.3295 | Val AUROC: 0.8592 | Val AUPRC: 0.8864
[04/20] Train Loss: 0.3078 | Val AUROC: 0.9090 | Val AUPRC: 0.9191
⭐ Modell gespeichert!
[05/20] Train Loss: 0.2924 | Val AUROC: 0.9317 | Val AUPRC: 0.9324
⭐ Modell gespeichert!
[06/20] Train Loss: 0.2792 | Val AUROC: 0.9351 | Val AUPRC: 0.9385
⭐ Modell gespeichert!
[07/20] Train Loss: 0.2702 | Val AUROC: 0.9222 | Val AUPRC: 0.9308
[08/20] Train Loss: 0.2633 | Val AUROC: 0.9124 | Val AUPRC: 0.9244
[09/20] Train Loss: 0.2564 | Val AUROC: 0.9296 | Val AUPRC: 0.9334
[10/20] Train Loss: 0.2511 | Val AUROC: 0.9215 | Val AUPRC: 0.9263
[11/20] Train Loss: 0.2468 | Val AUROC: 0.9297 | Val AUPRC: 0.9316
[12/20] Train Loss: 0.2432 | Val AUROC: 0.9389 | Val AUPRC: 0.9432
⭐ Modell gespeichert!
[13/20] Train Loss: 0.2398 | Val 